In [ ]:
from nbdev import *
%nbdev_default_export cli

Cells will be exported to RemoteConnect.cli,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# cli

> This is a cli module for RemoteConnect

In [ ]:
%nbdev_export
from nbdev.showdoc import *
from fastscript import call_parse, Param, bool_arg
from RemoteConnect.core import *

In [ ]:
%nbdev_export
@call_parse
def start_code(port:Param("Port to Start Code", type=int)=10000):
    "Start Code Server"
    print(f"Starting Code-Lib {port}")


In [ ]:
start_code()

Starting Code-Lib 10000


In [ ]:
say_hello("S3")

Say hello to S3


In [ ]:
start_code()

Starting Code-Lib 10000
